In [1]:
import torch
from torch.autograd import Variable
import pandas as pd
import numpy as np

In [2]:
inputSize = 3
outputSize = 1

In [3]:
class LinearRegression(torch.nn.Module):
    def __init__(self,inputSize,outputSize):
        super(LinearRegression, self).__init__()
        self.linear = torch.nn.Linear(inputSize,outputSize)
        
    def forward(self, x):
        out = self.linear(x)
        return out

In [4]:
model = LinearRegression(inputSize,outputSize)

In [5]:
state_dict = torch.load('./assets/checkpoint.pth')
print(state_dict.keys())

odict_keys(['linear.weight', 'linear.bias'])


In [6]:
model.load_state_dict(state_dict)

IncompatibleKeys(missing_keys=[], unexpected_keys=[])

In [7]:
test = torch.tensor([10,3,0])
test = test.type(torch.FloatTensor)
prediction = model(test)
print(prediction)

tensor([8.4987], grad_fn=<AddBackward0>)


# Let's Secure now

In [8]:
import torch as t
import syft as sy
hook = sy.TorchHook(t)

In [9]:
bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")
secure_worker =sy.VirtualWorker(hook, id="secure_worker")

In [10]:
from torch import nn
from torch import optim
import torch.nn.functional as F

In [11]:
encryptedModel = model.fix_precision().share(alice, bob, crypto_provider = secure_worker)

In [12]:
list(encryptedModel.parameters())

[Parameter containing:
 Parameter>FixedPrecisionTensor>(Wrapper)>[AdditiveSharingTensor]
 	-> (Wrapper)>[PointerTensor | me:94946845859 -> alice:40389828625]
 	-> (Wrapper)>[PointerTensor | me:18300260453 -> bob:38180226907]
 	*crypto provider: secure_worker*, Parameter containing:
 Parameter>FixedPrecisionTensor>(Wrapper)>[AdditiveSharingTensor]
 	-> (Wrapper)>[PointerTensor | me:53819435105 -> alice:43695678846]
 	-> (Wrapper)>[PointerTensor | me:46495423680 -> bob:17815363064]
 	*crypto provider: secure_worker*]

In [13]:
test

tensor([10.,  3.,  0.])

In [14]:
encryptedTest = test.fix_precision().share(alice, bob, crypto_provider = secure_worker)

In [15]:
encrypytedPrediction = encryptedModel(encryptedTest)

In [16]:
dynamicPrice = encrypytedPrediction.get().float_precision()

In [17]:
round(float(dynamicPrice.numpy()),3)

8.491

# Cheers!